In [1]:
import pandas as pd

In [2]:
microdata = "data/DADOS/MICRODADOS_ENEM_2017.csv"
data_dict  = "data/DICIONÁRIO/Dicionário_Microdados_Enem_2017.xlsx"

In [3]:
df = pd.read_csv(microdata, sep=';', chunksize=500, encoding='latin-1')
head = df.read(10)

In [4]:
head

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,...,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027
0,170003336736,2017,3503208,Araraquara,35,SP,29,F,0,1,...,A,C,B,B,C,B,B,B,A,A
1,170003333545,2017,5002902,Cassilândia,50,MS,22,F,0,1,...,A,B,A,A,C,B,A,A,A,A
2,170001663644,2017,3550308,São Paulo,35,SP,38,F,0,1,...,A,B,A,A,C,A,B,B,A,A
3,170001663645,2017,4209300,Lages,42,SC,35,F,0,1,...,B,C,A,B,D,A,B,B,A,A
4,170001663646,2017,2704302,Maceió,27,AL,40,M,0,3,...,A,B,B,A,C,A,C,B,A,A
5,170001663647,2017,4303103,Cachoeirinha,43,RS,20,F,0,1,...,B,D,B,A,D,A,C,B,A,A
6,170003334115,2017,4106902,Curitiba,41,PR,36,M,1,2,...,B,B,A,A,C,A,B,B,A,A
7,170003334116,2017,4214805,Rio do Sul,42,SC,18,F,0,1,...,B,B,A,B,D,B,B,B,A,A
8,170003334117,2017,4322509,Vacaria,43,RS,23,F,0,1,...,A,B,A,A,B,A,A,A,A,A
9,170001669940,2017,2607901,Jaboatão dos Guararapes,26,PE,23,M,0,3,...,A,B,B,A,D,B,B,B,A,A


# Data Description

## Student Information Data

- NU_INSCRICAO: Subscription Number
- NU_ANO: Year when the Test was Takenm
- CO_MUNICIPIO_RESIDENCIA: 

    Where does the student lives. It is strucutred as the following:
    - 1st digit: Region
    - 1st + 2nd digit: State
    - 3rd, 4, 5 and 6: City
    - 7: verification number
    
- NO_MUNICIPIO_RESIDENCIA: Residency City name

- CO_UF_RESIDENCIA: State Code 

- SG_UF_RESIDENCIA: State Code Name

- NU_IDADE: Age

- TP_SEXO: Gender

- TP_ESTADO_CIVIL: Civil State (Married, Divorced or Single)

- TP_COR_RACA: Ethnics

- TP_NACIONALIDADE: Nacionality

- CO_MUNICIPIO_NASCIMENTO:
    Student birthplace. It is strucutred as the following:
    - 1st digit: Region
    - 1st + 2nd digit: State
    - 3rd, 4, 5 and 6: City
    - 7: verification number
    
- NO_MUNICIPIO_NASCIMENTO: Birthplace City name

- CO_UF_NASCIMENTO: State Code 

- SG_UF_NASCIMENTO: State Code Name    

- TP_ST_CONCLUSAO: Which situation does student is with his or her studies

- TP_ANO_CONCLUIU: When does the student concluded his/her studies

- TP_ESCOLA: School Type (Private/Public/Foreign)

- TP_ENSINO: Institution type where does the student concluded his/her studies ()

- IN_TREINEIRO: Did the test just as a trainee 

## Scool Information Data

## Special Requisites Data

## Test aplication Location Data

## Data about the Test
### Test Type

### Student Grade
#### Objective Test
#### Essay Test

## Social Economic Data
